In [2]:
from gurobipy import *
import numpy as np
import pandas as pd
import math

#set model
m = Model("LRP")
#set variables
x1 = m.addVar(vtype='B', name="x1")
x2 = m.addVar(vtype='B', name="x2")
x3 = m.addVar(vtype='B', name="x3")
x4 = m.addVar(vtype='B', name="x4")
m.setAttr('ModelSense', GRB.MAXIMIZE)
#set constraint
m.addConstr(8*x1 + 2*x2 + x3 + 4*x4 <= 10, "c0")
count=0
df = pd.DataFrame(columns=['k','tk','lambda_k','v1','v2','x1','x2','x3','x4','Zd(v1,v2)','Value of Lagrangian solution'])
Z_star = 0
df.at[0,'lambda_k']=2
#iteration 1~10
for i in range(10):
    df.at[i,'k']=i+1
    if i == 0:
        df.at[i,'v1'] = 0
        df.at[i,'v2'] = 0
        m.setObjective((16-df.at[i,'v1'])*x1+(10-df.at[i,'v1'])*x2+
                       (-df.at[i,'v2'])*x3+(4-df.at[i,'v2'])*x4+df.at[i,'v1']+df.at[i,'v2'])
        m.update()
        m.setParam('OutputFlag', False )
        m.optimize()
        for v in m.getVars():
            df.at[i,v.varName] = v.x
        df.at[i,'Zd(v1,v2)'] = m.objVal
        if df.at[i,'x1']+df.at[i,'x2']<=1 and df.at[i,'x3']+df.at[i,'x4']<=1:
            df.at[i,'Value of Lagrangian solution'] = 16*df.at[i,'x1']+10*df.at[i,'x2']+4*df.at[i,'x4']
            Z_star = df.at[i,'Value of Lagrangian solution']
        else:
            df.at[i,'Value of Lagrangian solution']= "infeasible"
    else:
        m.setObjective((16-df.at[i,'v1'])*x1+(10-df.at[i,'v1'])*x2+
                       (-df.at[i,'v2'])*x3+(4-df.at[i,'v2'])*x4+df.at[i,'v1']+df.at[i,'v2'])
        m.update()
        m.setParam('OutputFlag', False )
        m.optimize()
        for v in m.getVars():
            df.at[i,v.varName] = v.x
        df.at[i,'Zd(v1,v2)'] = m.objVal
        if df.at[i,'Zd(v1,v2)'] >= df.at[i-1,'Zd(v1,v2)']:
            count = count+1
        #calculate value of Lagrangian solution
        if df.at[i,'x1']+df.at[i,'x2']<=1 and df.at[i,'x3']+df.at[i,'x4']<=1:
            df.at[i,'Value of Lagrangian solution'] = 16*df.at[i,'x1']+10*df.at[i,'x2']+4*df.at[i,'x4']
            Z_star = df.at[i,'Value of Lagrangian solution']
        else:
            df.at[i,'Value of Lagrangian solution']= "infeasible"
        #update lambda_k if needed
        if count >= 2:
            df.at[i,'lambda_k'] = df.at[i-1,'lambda_k']/2
            count=0
        else:
            df.at[i,'lambda_k'] = df.at[i-1,'lambda_k']
    #update tk
    df.at[i,'tk'] = df.at[i,'lambda_k']*(df.at[i,'Zd(v1,v2)']-Z_star)/(math.pow(1-df.at[i,'x1']-df.at[i,'x2'],2)+math.pow(1-df.at[i,'x3']-df.at[i,'x4'],2))
    #update next time v1 and v2a
    df.at[i+1,'v1'] = max(0,(df.at[i,'v1']-df.at[i,'tk']*(1-df.at[i,'x1']-df.at[i,'x2'])))
    df.at[i+1,'v2'] = max(0,(df.at[i,'v2']-df.at[i,'tk']*(1-df.at[i,'x3']-df.at[i,'x4'])))

In [3]:
pd.set_option('display.max_columns', None)
print(df)

      k    tk lambda_k    v1 v2   x1   x2   x3   x4 Zd(v1,v2)  \
0     1  26.0        2     0  0  1.0  1.0  0.0  0.0      26.0   
1     2  52.0        2  26.0  0  0.0  0.0  0.0  1.0      30.0   
2     3  22.0        2     0  0  1.0  1.0  0.0  0.0      26.0   
3     4  22.0      1.0  22.0  0  0.0  0.0  0.0  1.0      26.0   
4     5  11.0      1.0     0  0  1.0  1.0  0.0  0.0      26.0   
5     6   0.0      1.0  11.0  0  1.0  0.0  0.0  0.0      16.0   
6     7   0.0      0.5  11.0  0  1.0  0.0  0.0  0.0      16.0   
7     8   0.0      0.5  11.0  0  1.0  0.0  0.0  0.0      16.0   
8     9   0.0     0.25  11.0  0  1.0  0.0  0.0  0.0      16.0   
9    10   0.0     0.25  11.0  0  1.0  0.0  0.0  0.0      16.0   
10  NaN   NaN      NaN  11.0  0  NaN  NaN  NaN  NaN       NaN   

   Value of Lagrangian solution  
0                    infeasible  
1                           4.0  
2                    infeasible  
3                           4.0  
4                    infeasible  
5              

In [6]:
print("final_gap = %f"%((df.at[9,'Zd(v1,v2)']-Z_star)/Z_star*100)+"%")

final_gap = 0.000000%
